In [1]:
#----------- imports---------
import nltk #to import tokenize            
from nltk.corpus import stopwords #to remove stopwords
from nltk.tokenize import word_tokenize #tokenize text and lowercacse it
import os
from os import path # to check if the file is there or not 
from nltk.stem import PorterStemmer # to stem tokiens
import time
import re


In [3]:
def read_env(env_path):
    with open(env_path, 'r', encoding="utf-8") as env:
        env_settings = env.read()
        env_settings = re.sub(' ','',env_settings)
        env_settings = env_settings.split('\n')
        env_settings = [setting.split('=') for setting in env_settings]
    return dict(env_settings)

In [4]:
def settings(env_settings):
    language = env_settings['language']
    path = env_settings['path']
    type_files = env_settings['type_files']
    number_files= int(env_settings['number_files'])
    download= bool(int(env_settings['download']))
    return language,path,type_files,number_files,download

In [5]:
language,path,type_files,number_files,download=settings(read_env('env.txt'))
#-----------downlad------------
if download:
    nltk.download(['stopwords','punkt']) # if you don't have packages run it for one time

In [6]:
def scan_type_model():
    type_query=input('please enter 1 for Positional index model or 2 for Vector space model or 0 for exit: ')
    if (type_query=='1'):
        Positional_index_model()
    elif(type_query=='2'):
        # Positional_index_model()
        print('vector space model')
    elif(type_query=='0'):
        print('thanks :)')
        time.sleep(3)
    else:
        print('\t\tsorry put you entered wrong number try again')
        scan_type_model()

In [7]:
def get_ls_steming_tokins():
    inquery = input("Please enter a query:\n")
    query_to = tokenize(inquery)
    query =Stemming(query_to)
    return query,query_to

In [8]:
def Stemming(tokins):
    stemmer = PorterStemmer() 
    reviews_stem = [] 
    reviews_stem = [stemmer.stem(word) for word in tokins]
    return reviews_stem 

In [9]:
def tokenize(query): # get text and return list of tokens withot stopwords and lowercase 
    stopword = stopwords.words(language) #get list of stop words in language
    tokens = word_tokenize(query,language=language)  #get list of tokens
    tokens_without_stop_word = [word.lower() for word in tokens if not word in stopword] # remove stopwords
    return  tokens_without_stop_word

In [10]:
def positional (tokens): # get tokens and return
    pos_index = dict() 
    for index, term in enumerate(tokens): # index is int value for index and term is str value for token
        if term in pos_index:
            pos_index[term].append(index + 1)
        else:
            pos_index[term] = []
            pos_index[term].append(index + 1)
    return pos_index

In [11]:
def docID(plist): # get list and return number of doc in index 0 plist =[docID,[indices]]
    return plist[0]

In [12]:
def position(plist): # get list and return number of doc in index 0 plist =[docID,[indices]]
    return plist[1]

In [13]:
def pos_intersect(p1, p2, k): 
    answer = []
    len1 = len(p1)
    len2 = len(p2)
    i = j = 0
    while i != len1 and j != len2:  # p1 != null and p2 != null
        if docID(p1[i]) == docID(p2[j]):
            l = []  # l <- ()
            pp1 = position(p1[i])  # pp1 <- positions(p1) , p1[i]=[docID,[indices]]
            pp2 = position(p2[j])  # pp2 <- positions(p2) , p1[i]=[docID,[indices]]
            plen1 = len(pp1)
            plen2 = len(pp2)
            ii = jj = 0
            while ii != plen1:  # while (pp1 != null)
                while jj != plen2:  # while (pp2 != null)
                    if abs(pp1[ii] - pp2[jj]) == k:  # if (|index(pp1) - index(pp2)| == k) , k=1
                        l.append(pp2[jj]) 
                    elif pp2[jj] > pp1[ii]:  # index(pp2) > index(pp1)
                        break
                    jj += 1  # next(pp2)->int other index for second term
                answer.append([docID(p1[i]),  l])  #answer=[docID, index(first_term), ps_secondTerm]
                ii += 1  #  next(pp1)->int other index for first term
            i += 1  #  next(p1) ,p1=[docID,[indices]]
            j += 1  #  next(p2) ,p2=[docID,[indices]]
        elif docID(p1[i]) < docID(p2[j]):  # increace counter of term if his docID is smaller than the other
            i += 1  #  next(p1)
        else:
            j += 1  #  next(p2)
    return answer

In [14]:
def fileNames(path,type_str):
    file_names = [name for name in os.listdir(path) if name.endswith(type_str)]
    file_dirs=[]
    for name in file_names:
        file_dirs.append(path + '/' +name)
    return file_dirs,file_names

In [15]:
def Positional_index_model():
    positions = dict()
    files=list()
    finalresult = list()
    query,query_to=get_ls_steming_tokins()
    # print('\n',query,'\n')
    for term in query:
        positions[term]=list()
        positions[term].append(0)
        positions[term].append([])
    file_dirs,file_names =fileNames(path,type_files)
    for i in range(0,len(file_dirs)):
        with open(file_dirs[i], 'r', encoding="utf-8") as doc:
            read_string = doc.read()
            tokens=tokenize(read_string)
            tokens=Stemming(tokens)
            pos_index=positional(tokens)
            for word in query:
                for pos, term in enumerate(pos_index):
                    if term == word:
                        positions[word][1].append([i, pos_index[word]])
                        positions[word][0]=positions[word][0]+1
    # print(positions)
    ra=0
    for term in positions:
        print(query_to[ra],' :',positions[term]) #dict {term: [term_freq,[[doc1,[indices]],[doc2,[indices]]]]}
        ra +=1
    # print('len(q)->',len(query)) 
    if len(query)!=1:

        i=0
        j=0
        while i < len(query):
            j=i+1
            if j<len(query):
                # send positions without freq positions[query][1]->[[doc1,[inindices],[doc2,[inindices],...]]
                files.append(pos_intersect(positions[query[i]][1], positions[query[j]][1], 1)) #get intersect of two terms in query
            i=i+1
        b = 1
        lfiles =len(files)  #files=[[[doc1,index],[doc2,index]],[[doc1,index],[doc2,index]],....]
        #  -###################     call pos_intersect on the output of the last loop 
        while b<lfiles: #loop N-1 time on intersect of lists 
            i=0
            j=0
            while i < len(files):#files=[[[doc1,index],[doc2,index]],[[doc1,index],[doc2,index]],....]
                j=i+1
                if j<len(files):
                    finalresult.append(pos_intersect(files[i], files[j], 1))
                i=i+1
            files=finalresult.copy()
            finalresult=list()
            b +=1
        #######################
        if files!=[]:
            docs=set([i[0] for fi in files for i in fi if i[1]!=[]])
            for i in docs:print(file_names[i])
            scan_type_model()
        else :
            print('this query dosen\'t match ')
            scan_type_model()
    else:
        # for i in positions[query[0]][1]:
            # print(i)
        files=[docID(i) for i in positions[query[0]][1]]
        if files !=[]:
            for i in set(files):print(file_names[i])
            scan_type_model()
        else :
            print('this query dosen\'t match ')
            scan_type_model()

    


In [16]:
scan_type_model()

ahmed  : [7, [[0, [1, 17]], [1, [1]], [2, [19]], [6, [1]], [7, [19]], [8, [1]], [9, [19]]]]
aly  : [8, [[0, [6]], [1, [5]], [2, [20]], [3, [5]], [6, [6]], [7, [20]], [8, [6]], [9, [20]]]]
andrew  : [7, [[0, [5]], [1, [16]], [2, [21]], [3, [6]], [6, [5]], [7, [21]], [8, [5]]]]
doc2.txt
doc7.txt
thanks :)
